In [ ]:
!nvidia-smi -L

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# Import necessary Libraries
import numpy as np
import os
import scipy.stats as st
import math
from sklearn import preprocessing
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
from math import exp
from math import log
import tensorflow as tf

import keras
from keras import layers
# from keras.utils import to_categorical
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
from keras.layers import Input, Dropout, Add, Dense 
from keras.layers import MaxPooling1D, Reshape, Activation
from keras.layers import BatchNormalization, Flatten, Conv1D


def DataPreparation_DNN(data_input_file):
  data = np.load(data_input_file, allow_pickle=True)
  X = data['X']
  X = X[:, 0, :, :]
  
  Y = data['y']
  n_class = Y.shape[1]
  folds = data['folds']
  # Y = np.argmax(Y, axis=1)
 
  return X,Y,folds,n_class


# Build ResNet Model
def identity_block(input, kernel_size, filters, stage, block):
  # Variables
  filters1, filters2, filters3 = filters
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'
  # Create layers
  output = keras.layers.Conv1D(filters1, 1, kernel_initializer='he_normal', name=conv_name_base + '2a')(input)
  output = keras.layers.BatchNormalization(name=bn_name_base + '2a')(output)
  output = keras.layers.Activation('relu')(output)
  output = keras.layers.Conv1D(filters2, kernel_size, padding='same', kernel_initializer='he_normal', name=conv_name_base + '2b')(output)
  output = keras.layers.BatchNormalization(name=bn_name_base + '2b')(output)
  output = keras.layers.Activation('relu')(output)
  output = keras.layers.Conv1D(filters3, 1, kernel_initializer='he_normal', name=conv_name_base + '2c')(output)
  output = keras.layers.BatchNormalization(name=bn_name_base + '2c')(output)
  output = keras.layers.add([output, input])
  output = keras.layers.Activation('relu')(output)
  # Return a block
  return output


def conv_block(input, kernel_size, filters, stage, block, strides=2):
  # Variables
  filters1, filters2, filters3 = filters
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'
  # Create block layers
  output = keras.layers.Conv1D(filters1, 1, strides=strides, kernel_initializer='he_normal', name=conv_name_base + '2a')(input)
  output = keras.layers.BatchNormalization(name=bn_name_base + '2a')(output)
  output = keras.layers.Activation('relu')(output)
  output = keras.layers.Conv1D(filters2, kernel_size, padding='same', kernel_initializer='he_normal', name=conv_name_base + '2b')(output)
  output = keras.layers.BatchNormalization(name=bn_name_base + '2b')(output)
  output = keras.layers.Activation('relu')(output)
  output = keras.layers.Conv1D(filters3, 1, kernel_initializer='he_normal', name=conv_name_base + '2c')(output)
  output = keras.layers.BatchNormalization(name=bn_name_base + '2c')(output)
  shortcut = keras.layers.Conv1D(filters3, 1, strides=strides, kernel_initializer='he_normal', name=conv_name_base + '1')(input)
  shortcut = keras.layers.BatchNormalization(name=bn_name_base + '1')(shortcut)
  output = keras.layers.add([output, shortcut])
  output = keras.layers.Activation('relu')(output)
  # Return a block
  return output



def build_model_ResNet(row, col, num_classes):
  input_shape = (row,col)
  # Create an input layer 
  input = keras.layers.Input(shape=input_shape)
  # Create output layers
  # output = keras.layers.ZeroPadding2D(padding=2, name='padding_conv1')(input)
  output = keras.layers.Conv1D(64, 7, strides=2, use_bias=False, name='conv1')(input)
  output = keras.layers.BatchNormalization(name='bn_conv1')(output)
  output = keras.layers.Activation('relu', name='conv1_relu')(output)
  output = keras.layers.MaxPooling1D(3, strides=2, padding='same', name='pool1')(output)
  output = conv_block(output, 3, [64, 64, 256], stage=2, block='a', strides=1)
  output = identity_block(output, 3, [64, 64, 256], stage=2, block='b')
  output = identity_block(output, 3, [64, 64, 256], stage=2, block='c')
  output = conv_block(output, 3, [128, 128, 512], stage=3, block='a')
  output = identity_block(output, 3, [128, 128, 512], stage=3, block='b')
  output = identity_block(output, 3, [128, 128, 512], stage=3, block='c')
  output = identity_block(output, 3, [128, 128, 512], stage=3, block='d')
  output = conv_block(output, 3, [256, 256, 1024], stage=4, block='a')
  output = identity_block(output, 3, [256, 256, 1024], stage=4, block='b')
  output = identity_block(output, 3, [256, 256, 1024], stage=4, block='c')
  output = identity_block(output, 3, [256, 256, 1024], stage=4, block='d')
  output = identity_block(output, 3, [256, 256, 1024], stage=4, block='e')
  output = identity_block(output, 3, [256, 256, 1024], stage=4, block='f')
  output = conv_block(output, 3, [512, 512, 2048], stage=5, block='a')
  output = identity_block(output, 3, [512, 512, 2048], stage=5, block='b')
  output = identity_block(output, 3, [512, 512, 2048], stage=5, block='c')
  output = keras.layers.GlobalAveragePooling1D(name='pool5')(output)
  output = keras.layers.Dense(num_classes, activation='softmax', name='fc1000')(output)
  # Create a model from input layer and output layers
  model = keras.models.Model(inputs=input, outputs=output)
  # # Print model
  # print()
  # print(model.summary(), '\n')
  # Compile the model
  # Return a model
  return model


batch_size = 128
epochs = 100
learning_rate = 0.001


def Train_ResNet(X, y, folds):
  avg_acc = []
  avg_recall = []
  avg_f1 = []

  n_class = y.shape[1]  
  _,img_rows, img_cols = X.shape
  X=X.reshape(X.shape[0],img_rows,img_cols,1)
  _,img_rows, img_cols,_ = X.shape
    
  for i in range(3, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]
    X_train = X[train_idx]
    y_train = y[train_idx]
    X_test = X[test_idx]
    y_test = y[test_idx]

    model=build_model_ResNet(img_rows,img_cols,n_class)
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='ADAM')
    model.fit(X_train, y_train, batch_size=192, epochs=150,verbose=2)
    y_pred = model.predict(X_test)

    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y[test_idx], axis=1)
    acc_fold = accuracy_score(y_true, y_pred)
    avg_acc.append(acc_fold)
    recall_fold = recall_score(y_true, y_pred, average='macro')
    avg_recall.append(recall_fold)
    f1_fold = f1_score(y_true, y_pred, average='macro')
    avg_f1.append(f1_fold)
    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
    print('________________________________________________________________')
        
  print('Overall Accuracy[{:.4f}] Overall Recall[{:.4f}] Overall F1[{:.4f}]'.format(np.mean(avg_acc), np.mean(avg_recall), np.mean(avg_f1)))
  print('________________________________________________________________')

  

if __name__ == "__main__":  
  X,y,folds,n_class = DataPreparation_DNN('/content/drive/MyDrive/SNOW/USCHAD.npz')

  Train_ResNet(X, y, folds)

  